In [12]:
#@title Run this cell to generate the data
! pip install faker
import numpy as np
import pandas as pd
import uuid
import datetime

from faker.providers.person.en import Provider
from faker import Faker

from faker import Faker
fake = Faker()
Faker.seed(0)

n_employees = 50
first_names = pd.Series(list(set(Provider.first_names))).sample(n_employees)
df_employees = pd.DataFrame(columns=["id", "name", "salary"])
df_employees["name"] = (first_names).sample(frac=1).values
mu = 28_000
minimum = 16_000
df_employees["salary"] = (np.random.pareto(a=6, size=(50,1)) * mu) + minimum
df_employees["salary"] = (df_employees["salary"]/100).astype(int) * 100
df_employees["id"] = pd.Series([str(uuid.uuid4()).split('-')[1] for x in range(n_employees)])

def create_random_project(number_of_employees, duration, year, is_ongoing=False):
    df_projects = pd.DataFrame(index=range(number_of_employees), columns=["employee_id", "project_id", "start_date", "end_date"])
    
    employees = df_employees.id.sample(number_of_employees)
    
    start_date = datetime.date(year=year, month=1, day=1)
    end_date = datetime.date(year=year+1, month=1, day=1)
    project_start_date = fake.date_between(start_date=start_date, end_date=end_date)
    project_end_date = project_start_date + datetime.timedelta(days=duration)
    
    if is_ongoing:
        project_end_date = None
        
    project_id = str(uuid.uuid4()).split('-')[4]
    
    df_projects["employee_id"] = employees.values
    df_projects["project_id"] = project_id
    df_projects["start_date"] = project_start_date
    df_projects["end_date"] = project_end_date
    
    
    return df_projects

n_projects = 150
projects = []
for proj in range(n_projects):
    number_of_employees = np.random.choice([1,2,3,4], replace=True, p=[0.25, 0.5, 0.15, 0.1], size=1)
    duration = (np.random.pareto(a=5, size=1) * 30*4) + 14
    year = np.random.choice([2017, 2018, 2019, 2020])
    is_ongoing = bool(np.random.binomial(n=1, p=0.4)) if year >= 2019 else False
    new_project = (create_random_project(number_of_employees=number_of_employees[0], duration=duration[0], year=year, is_ongoing=is_ongoing).values)
    
    projects.append(new_project)
    
projects_data = np.vstack(projects)
df_projects = (
    pd.DataFrame(projects_data, columns=["employee_id", "project_id", "start_date", "end_date"])
    .sort_values(["project_id", "start_date", "end_date"])
    .reset_index(drop=True)
)

In [13]:
df_m = df_employees.merge(df_projects, left_on='id', right_on='employee_id')
grouped = df_m.groupby(['id','name'],as_index=False).agg({'project_id':'count','salary':'min'})
grouped[grouped['project_id'] >=10].sort_values('salary',ascending=True).nsmallest(3,'salary')

,id,name,project_id,salary
38,cc9b,Sheilah,10,16100
32,a287,Pauletta,11,16400
42,da12,Diego,10,17200
